In [1]:
import pandas as pd
import datetime
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [2]:
minedex = pd.read_csv('../../data/recent_mindex.csv')
minedex.shape

(1222, 13)

In [3]:
# Acess Site.csv for lat and lon.
site_locs = pd.read_excel('../../data/minedex/Sites.xlsx')
site_locs = site_locs[['SiteCode', 'Latitude', 'Longitude']].set_index('SiteCode')
site_locs = site_locs.rename(columns={
    "Latitude": "lat",
    "Longitude": "lon"
})

In [4]:
# minedex = minedex.merge(site_locs, left_on="SiteCode", right_index=True)

In [5]:
full_to_atomic = {'Hydrogen' : 'H', 'Helium' : 'He', 'Lithium' : 'Li', 'Beryllium' : 'Be', 'Boron' : 'B', 'Carbon' : 'C', 'Nitrogen' : 'N', 'Oxygen' : 'O', 'Fluorine' : 'F', 'Neon' : 'Ne', 'Sodium' : 'Na', 'Magnesium' : 'Mg', 'Aluminium' : 'Al', 'Silicon' : 'Si', 'Phosphorus' : 'P', 'Sulfur' : 'S', 'Chlorine' : 'Cl', 'Argon' : 'Ar', 'Potassium' : 'K', 'Calcium' : 'Ca', 'Scandium' : 'Sc', 'Titanium' : 'Ti', 'Vanadium' : 'V', 'Chromium' : 'Cr', 'Manganese' : 'Mn', 'Iron' : 'Fe', 'Cobalt' : 'Co', 'Nickel' : 'Ni', 'Copper' : 'Cu', 'Zinc' : 'Zn', 'Gallium' : 'Ga', 'Germanium' : 'Ge', 'Arsenic' : 'As', 'Selenium' : 'Se', 'Bromine' : 'Br', 'Krypton' : 'Kr', 'Rubidium' : 'Rb', 'Strontium' : 'Sr', 'Yttrium' : 'Y', 'Zirconium' : 'Zr', 'Niobium' : 'Nb', 'Molybdenum' : 'Mo', 'Technetium' : 'Tc', 'Ruthenium' : 'Ru', 'Rhodium' : 'Rh', 'Palladium' : 'Pd', 'Silver' : 'Ag', 'Cadmium' : 'Cd', 'Indium' : 'In', 'Tin' : 'Sn', 'Antimony' : 'Sb', 'Tellurium' : 'Te', 'Iodine' : 'I', 'Xenon' : 'Xe', 'Cesium' : 'Cs', 'Barium' : 'Ba', 'Lanthanum' : 'La', 'Cerium' : 'Ce', 'Praseodymium' : 'Pr', 'Neodymium' : 'Nd', 'Promethium' : 'Pm', 'Samarium' : 'Sm', 'Europium' : 'Eu', 'Gadolinium' : 'Gd', 'Terbium' : 'Tb', 'Dysprosium' : 'Dy', 'Holmium' : 'Ho', 'Erbium' : 'Er', 'Thulium' : 'Tm', 'Ytterbium' : 'Yb', 'Lutetium' : 'Lu', 'Hafnium' : 'Hf', 'Tantalum' : 'Ta', 'Tungsten' : 'W', 'Rhenium' : 'Re', 'Osmium' : 'Os', 'Iridium' : 'Ir', 'Platinum' : 'Pt', 'Gold' : 'Au', 'Mercury' : 'Hg', 'Thallium' : 'Tl', 'Lead' : 'Pb', 'Bismuth' : 'Bi', 'Polonium' : 'Po', 'Astatine' : 'At', 'Radon' : 'Rn', 'Francium' : 'Fr', 'Radium' : 'Ra', 'Actinium' : 'Ac', 'Thorium' : 'Th', 'Protactinium' : 'Pa', 'Uranium' : 'U', 'Neptunium' : 'Np', 'Plutonium' : 'Pu', 'Americium' : 'Am', 'Curium' : 'Cm', 'Berkelium' : 'Bk', 'Californium' : 'Cf', 'Einsteinium' : 'Es', 'Fermium' : 'Fm', 'Mendelevium' : 'Md', 'Nobelium' : 'No', 'Lawrencium' : 'Lr', 'Rutherfordium' : 'Rf', 'Dubnium' : 'Db', 'Seaborgium' : 'Sg', 'Bohrium' : 'Bh', 'Hassium' : 'Hs', 'Meitnerium' : 'Mt', 'Darmstadtium' : 'Ds', 'Roentgenium' : 'Rg', 'Copernicium' : 'Cn', 'Nihonium' : 'Nh', 'Flerovium' : 'Fl', 'Moscovium' : 'Mc', 'Livermorium' : 'Lv', 'Tennessine' : 'Ts', 'Og' : 'Oganesson',}

def convert_to_atomic(col):
    ### NEED TO CATER TO EDGE CASES
    ### e.g Mag Oxide, Alumina etc.
    ###
    full_name = col
    
    if full_name in full_to_atomic.keys():
        atomic = full_to_atomic[full_name]
        return atomic

    else:
        return np.nan
        

minedex['CommodityName'] = minedex['CommodityName'].apply(convert_to_atomic)

In [6]:
### DROPPING WIERD COMMODS FOR NOW.
minedex = minedex[~minedex['CommodityName'].isna()]

In [7]:
# get list of unique mineral combinations
list_combinations = minedex.groupby('SiteCode')['CommodityName'].unique().to_list()

# sort alphabetically
list_combinations = [sorted(commods) for commods in list_combinations]

unique_commods = []
for commods in list_combinations:
    if commods not in unique_commods:
        unique_commods.append(commods)

In [8]:
# WAMEX CLEANING
def replace_neg9999(val):
    if val == -9999: 
        return 0
    else:
        return val
    
    
def abs_halve_neg_val(val):
    if val < 0: 
        return abs(val) / 2
    else: 
        return val

def clean_assays(df):
    df = df.applymap(replace_neg9999)
    df = df.applymap(abs_halve_neg_val)

    # all 9999 -> 0 values are removed
    indexs_0 = df[df.values == 0].index
    df = df.drop(index=indexs_0)
    
    return df

def abundant_ratio(row, abundant_mineral):
    return row / row[abundant_mineral]

def make_pickle_fn(df, model, prefix='../../pickles/'):
    fn = f'{prefix}{type(model).__name__}-'
    for i in df.columns:
        fn = fn + i + '-'

    fn = fn[:-1] + '.pickle'   
    return fn

def filter_by_wamex_minerals(unique_commods, wamex_unique_minerals):
    # filter unique commods by availability in wamex assays
    filtered_unique_commods = []
    for commods in unique_commods:
        filtered_commods = []
        for commod in commods:
            if commod in wamex_unique_minerals:
                filtered_commods.append(commod)
        filtered_unique_commods.append(filtered_commods)
    return filtered_unique_commods

In [9]:
# WAMEX
wamex = pd.read_csv('../../asher/Merged_Ni_lessthan15km.csv',index_col=0)
# wamex.head()

In [10]:
wamex = wamex.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'lat', 'long', 
                     'minedex_coords', 'minedex_site',
                    'distance'])

wamex = wamex.rename(columns={
    "wamex_lat": "lat",
    "wamex_lon": "lon"
})

wamex = wamex.drop_duplicates()

In [11]:
wamex_minerals = wamex.drop(columns=['lat', 'lon'])

wamex.loc[:, wamex_minerals.columns] = clean_assays(wamex.loc[:,wamex_minerals.columns])

# Ratio transform
wamex.loc[:, wamex_minerals.columns] = wamex.loc[:, wamex_minerals.columns].apply(
                                                            abundant_ratio, abundant_mineral='Ni',
                                                            axis=1)

# Available minerals in WAMEX
wamex_unique_minerals = wamex_minerals.columns
filtered_unique_commods = filter_by_wamex_minerals(unique_commods, wamex_unique_minerals)

In [12]:
all_unique_commods = []
for cl in filtered_unique_commods:
    all_unique_commods.extend(cl)

In [13]:
wamex_commods = list(set(all_unique_commods))
wamex_commods

['Ni', 'S', 'Fe', 'Ag', 'Au', 'Pt', 'Pd', 'Co', 'Rh', 'Cu']

In [14]:
minedex = minedex[['SiteCode', 'Date', 'CommodityName', 'ppm']]

# sum up duplicate ppm values 
minedex['ppm'] = minedex.groupby(['SiteCode', 'CommodityName'])['ppm'].transform('sum')

# drop duplicate ppm values.
minedex = minedex.drop_duplicates()

In [15]:
# Filter by WAMEX available commods
minedex = minedex[minedex['CommodityName'].isin(wamex_commods)]

In [16]:
minedex_piv = minedex[['SiteCode', 'CommodityName', 'ppm']]

In [17]:
unique_idxs = minedex_piv[['SiteCode', 'CommodityName']].drop_duplicates().index

# don't know why these are here.
minedex_piv[~minedex_piv.index.isin(unique_idxs)]
minedex_piv = minedex_piv[minedex_piv.index.isin(unique_idxs)]

### Prediction Set

In [18]:
# create a pivot table by SiteCode
minedex_piv = minedex_piv.pivot(index='SiteCode', columns='CommodityName', values='ppm').rename_axis(None)

In [19]:
minedex_piv.columns = minedex_piv.columns.rename(None)

In [20]:
# stupid af workaround to get merge to work.
minedex_piv = minedex_piv.reset_index().merge(site_locs, left_on='index', right_index=True)
minedex_piv

,index,Ag,Au,Co,Cu,Fe,Ni,Pd,Pt,Rh,S,lat,lon
0,S0000192,NaN,NaN,NaN,NaN,NaN,59000.0,NaN,NaN,NaN,NaN,-32.416982,119.689058
1,S0000539,NaN,NaN,NaN,NaN,NaN,20000.0,NaN,NaN,NaN,NaN,-31.011440,121.680000
2,S0001174,NaN,NaN,500.0,53.64,237400.0,15000.0,NaN,NaN,NaN,NaN,-33.633382,120.374303
3,S0001178,NaN,NaN,NaN,NaN,NaN,18500.0,NaN,NaN,NaN,NaN,-30.393502,121.648875
4,S0001188,NaN,NaN,NaN,NaN,NaN,16500.0,NaN,NaN,NaN,NaN,-30.736222,121.802770
...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,S0236620,NaN,NaN,2300.0,NaN,NaN,10900.0,NaN,NaN,NaN,NaN,-32.095677,121.644431
326,S0236621,NaN,NaN,1100.0,NaN,NaN,4500.0,NaN,NaN,NaN,NaN,-32.201621,121.664588
327,S0236838,NaN,NaN,700.0,9000.00,NaN,14000.0,NaN,NaN,NaN,NaN,-31.591497,123.441046
327,S0236838,NaN,NaN,700.0,9000.00,NaN,14000.0,NaN,NaN,NaN,NaN,-31.591497,123.441046


In [21]:
# for each site, make a string of non-null minerals. 
# template: LinearRegression-Ag-Au-Co-Cu-Pd-Pt-lat-lon

minedex_minerals = minedex_piv.drop(columns=['index', 'lat', 'lon'])

def commod_string(row):
    commod_str = "-".join(row.dropna().index.to_list()).replace('-Co', "").replace('Co', "")
    if commod_str.startswith('-'):
        commod_str = commod_str[1:]
    
    row['commod_str'] = commod_str
    return row

minedex_minerals_str = minedex_minerals.apply(commod_string, axis=1)

In [22]:
"Ni-Cu".replace('Co', "")

'Ni-Cu'

In [23]:
minedex_minerals_str['SiteCode'] = minedex_piv['index']
minedex_minerals_str

,Ag,Au,Co,Cu,Fe,Ni,Pd,Pt,Rh,S,commod_str,SiteCode
0,NaN,NaN,NaN,NaN,NaN,59000.0,NaN,NaN,NaN,NaN,Ni,S0000192
1,NaN,NaN,NaN,NaN,NaN,20000.0,NaN,NaN,NaN,NaN,Ni,S0000539
2,NaN,NaN,500.0,53.64,237400.0,15000.0,NaN,NaN,NaN,NaN,Cu-Fe-Ni,S0001174
3,NaN,NaN,NaN,NaN,NaN,18500.0,NaN,NaN,NaN,NaN,Ni,S0001178
4,NaN,NaN,NaN,NaN,NaN,16500.0,NaN,NaN,NaN,NaN,Ni,S0001188
...,...,...,...,...,...,...,...,...,...,...,...,...
325,NaN,NaN,2300.0,NaN,NaN,10900.0,NaN,NaN,NaN,NaN,Ni,S0236620
326,NaN,NaN,1100.0,NaN,NaN,4500.0,NaN,NaN,NaN,NaN,Ni,S0236621
327,NaN,NaN,700.0,9000.00,NaN,14000.0,NaN,NaN,NaN,NaN,Cu-Ni,S0236838
327,NaN,NaN,700.0,9000.00,NaN,14000.0,NaN,NaN,NaN,NaN,Cu-Ni,S0236838


In [24]:
# Add lat and lon
minedex_min_loc = minedex_minerals_str.merge(site_locs, left_on="SiteCode", right_index=True)

In [25]:
minerals_df = minedex_min_loc.drop(columns=['lat', 'lon', 'commod_str', 'SiteCode'])

orig_ni = minerals_df['Ni']

#Ratio transform
# minedex_min_loc.loc[:, minerals_df.columns] = minedex_min_loc.loc[:, minerals_df.columns].apply(
#                                                             abundant_ratio, abundant_mineral='Ni', axis=1)
# apply Log transform
minedex_min_loc.loc[:, minerals_df.columns] = minedex_min_loc.loc[:,minerals_df.columns].apply(np.log, axis=1)

In [26]:
# keep ref of sitecode to index
site_codes_ref = minedex_min_loc['SiteCode']

In [27]:
def load_commod_model(commod_str, model='LinearRegression'):
    with open(f'../../pickles/{model}-{commod_str}-lat-lon.pickle', 'rb') as file:
        m = pickle.load(file)    
    return m

### Filter by lotsa minersals

In [36]:
minedex_min_loc['commod_str'].str.len().sort_values(ascending=True)

0       2
214     2
212     2
211     2
210     2
       ..
179    17
193    17
179    17
193    17
310    17
Name: commod_str, Length: 380, dtype: int64

In [39]:
minedex_min_loc.loc[:,minedex.index == 17]

IndexError: Boolean index has wrong length: 568 instead of 14

In [28]:
# create a list of dataframes by unique commodity groups.
gb = minedex_min_loc.groupby('commod_str')    
list_of_commod_dfs = [gb.get_group(x) for x in gb.groups]

In [280]:
def create_site_pred(commod_group_df):
    '''Gets a dataframe containing a unique commodity group,
    '''
    # drop empty mineral cols
    commod_group_df = commod_group_df.dropna(axis=1)
    # dont know why duplicates
    commod_group_df = commod_group_df.drop_duplicates()
    # create X vars from data.
#     commod_group_df_X = commod_group_df.drop(columns=['commod_str', 'SiteCode', 'Ni'])
    commod_group_df_X = commod_group_df.drop(columns=['commod_str', 'SiteCode'])
    # if site already has a Cobalt value, drop.
    if 'Co' in commod_group_df_X.columns:
        commod_group_df_X = commod_group_df_X.drop(columns=['Co'])
    
    commod_str = commod_group_df['commod_str'].iloc[0]
    
    try:
        # use first commod string to find relevant model pickle.
        # Linear Regression
        lr = load_commod_model(commod_str=commod_str, model='LinearRegression')
        lr_preds = lr.predict(commod_group_df_X)
        commod_group_df['Co_pred_lr'] = lr_preds
        # Random Forest
        rf = load_commod_model(commod_str=commod_str, model='RandomForestRegressor')
        rf_preds = rf.predict(commod_group_df_X)
        commod_group_df['Co_pred_rf'] = rf_preds
        
        # make commod_str column
        commod_group_df['commods_used'] = commod_str
            
        site_pred_df = commod_group_df[['SiteCode', 'Co_pred_lr', 'Co_pred_rf', 'commods_used']]

        return site_pred_df
    
    except:
        print(f'Could not find models for {commod_str}')

In [281]:
list_of_site_pred_dfs = []

for commod_group_df in list_of_commod_dfs:
    site_pred_df = create_site_pred(commod_group_df)
    list_of_site_pred_dfs.append(site_pred_df)

In [282]:
all_site_preds_df = pd.concat(list_of_site_pred_dfs, axis=0)

In [283]:
# frik knows why the merge creates duplicates!!!!
all_site_preds_df = all_site_preds_df.merge(minedex[['SiteCode', 'Date']], on='SiteCode').drop_duplicates()

In [284]:
# get original Ni values for inveser transform
# all_site_preds_df = all_site_preds_df.merge(orig_ni, left_index=True, right_index=True).drop_duplicates()

### Invert ratio and log

In [285]:
all_site_preds_df['Co_pred_lr'] = np.exp(all_site_preds_df['Co_pred_lr'])  #* all_site_preds_df['Ni'] 
all_site_preds_df['Co_pred_rf'] = np.exp(all_site_preds_df['Co_pred_rf'])  #* all_site_preds_df['Ni'] 

In [286]:
all_site_preds_df

,SiteCode,Co_pred_lr,Co_pred_rf,commods_used,Date
0,S0230165,235.908886,365.299343,Ag-Au-Cu-Ni,22/06/2020
4,S0018904,697.988768,342.184467,Ag-Au-Cu-Ni-Pd-Pt,09/12/2020
11,S0019102,649.922267,318.765346,Ag-Au-Cu-Ni-Pd-Pt,09/12/2020
18,S0001805,1068.182673,644.264130,Au-Cu-Ni-Pd-Pt,30/06/2015
23,S0004096,170.088795,106.824047,Au-Cu-Ni-Pd-Pt,30/11/2020
28,S0032888,196.788142,101.802763,Au-Cu-Ni-Pd-Pt,04/03/2019
33,S0001806,73.391661,96.060004,Au-Cu-Ni-Pd-Pt-Rh,26/09/2008
39,S0230680,73.391661,96.060004,Au-Cu-Ni-Pd-Pt-Rh,26/09/2008
45,S0001174,314.747747,1733.726338,Cu-Fe-Ni,31/12/2017
49,S0005631,277.514417,1331.178179,Cu-Fe-Ni,31/12/2017


In [287]:
all_site_preds_df.merge(true_cobalt)

,SiteCode,Co_pred_lr,Co_pred_rf,commods_used,Date,ppm
0,S0018904,697.988768,342.184467,Ag-Au-Cu-Ni-Pd-Pt,09/12/2020,410.0
1,S0019102,649.922267,318.765346,Ag-Au-Cu-Ni-Pd-Pt,09/12/2020,360.0
2,S0001174,314.747747,1733.726338,Cu-Fe-Ni,31/12/2017,500.0
3,S0005631,277.514417,1331.178179,Cu-Fe-Ni,31/12/2017,600.0
4,S0006586,192.061664,283.708294,Cu-Fe-Ni,31/12/2012,200.0
5,S0006685,363.996006,1439.636026,Cu-Fe-Ni,31/12/2012,800.0
6,S0229459,172.210609,250.131672,Cu-Fe-Ni,31/12/2015,200.0
7,S0020966,266.960070,322.827527,Cu-Fe-Ni-S,13/11/2019,500.0
8,S0224398,56.328326,70.725449,Cu-Fe-Ni-S,24/02/2015,130.0
9,S0001442,671.090123,1199.021306,Cu-Ni,02/02/2006,1000.0


In [288]:
pd.set_option('display.max_rows', 500)

# add cobalt on
true_cobalt = minedex[minedex['CommodityName'] == 'Co'][['SiteCode', 'ppm']]
all_site_preds_df = all_site_preds_df.merge(true_cobalt, on='SiteCode')
all_site_preds_df = all_site_preds_df.rename(columns={'ppm': 'Co_true'})
# all_site_preds_df

In [289]:
all_site_preds_df = all_site_preds_df[['SiteCode', 'Date', 'commods_used', 'Co_pred_lr', 'Co_pred_rf', 'Co_true']]
all_site_preds_df

,SiteCode,Date,commods_used,Co_pred_lr,Co_pred_rf,Co_true
0,S0018904,09/12/2020,Ag-Au-Cu-Ni-Pd-Pt,697.988768,342.184467,410.0
1,S0019102,09/12/2020,Ag-Au-Cu-Ni-Pd-Pt,649.922267,318.765346,360.0
2,S0001174,31/12/2017,Cu-Fe-Ni,314.747747,1733.726338,500.0
3,S0005631,31/12/2017,Cu-Fe-Ni,277.514417,1331.178179,600.0
4,S0006586,31/12/2012,Cu-Fe-Ni,192.061664,283.708294,200.0
5,S0006685,31/12/2012,Cu-Fe-Ni,363.996006,1439.636026,800.0
6,S0229459,31/12/2015,Cu-Fe-Ni,172.210609,250.131672,200.0
7,S0020966,13/11/2019,Cu-Fe-Ni-S,266.960070,322.827527,500.0
8,S0224398,24/02/2015,Cu-Fe-Ni-S,56.328326,70.725449,130.0
9,S0001442,02/02/2006,Cu-Ni,671.090123,1199.021306,1000.0


In [296]:
all_site_preds_df.to_csv('../../data/preds_lr_rf.csv')

In [293]:
plot_preds_df = all_site_preds_df[['Co_pred_lr', 'Co_pred_rf', 'Co_true', 'commods_used']].set_index('commods_used')


In [295]:
plot_preds_df

,Co_pred_lr,Co_pred_rf,Co_true
commods_used,,,
Ag-Au-Cu-Ni-Pd-Pt,697.988768,342.184467,410.0
Ag-Au-Cu-Ni-Pd-Pt,649.922267,318.765346,360.0
Cu-Fe-Ni,314.747747,1733.726338,500.0
Cu-Fe-Ni,277.514417,1331.178179,600.0
Cu-Fe-Ni,192.061664,283.708294,200.0
Cu-Fe-Ni,363.996006,1439.636026,800.0
Cu-Fe-Ni,172.210609,250.131672,200.0
Cu-Fe-Ni-S,266.960070,322.827527,500.0
Cu-Fe-Ni-S,56.328326,70.725449,130.0


In [298]:
err_df = pd.read_csv('../../data/model_errors.csv', index_col=0)

,lr_path,lr_train_mse,lr_test_mse,rf_train_mse,rf_test_mse
0,-Ni-lat-lon,1.002014,0.982939,0.063071,0.425260
1,-Ni-lat-lon,1.002014,0.982939,0.062551,0.427272
2,-Cu-Fe-Ni-lat-lon,0.907761,0.931356,0.054931,0.380916
3,-Cu-Ni-lat-lon,0.951613,0.866928,0.056825,0.396301
4,-Cu-Ni-S-lat-lon,1.034794,0.915617,0.061073,0.412625
5,-Ni-lat-lon,1.002014,0.982939,0.062859,0.425550
6,-Cu-Ni-lat-lon,0.951613,0.866928,0.056927,0.403983
7,-Au-Cu-Ni-Pd-Pt-lat-lon,0.524660,0.534778,0.045811,0.323079
8,-Au-Cu-Ni-Pd-Pt-Rh-lat-lon,0.556201,0.418707,0.074973,0.394984
9,-Fe-Ni-lat-lon,0.940294,0.997591,0.054887,0.413955


In [302]:
def clean_lr_path(row):
    if row.startswith('-'):
        row = row[1:]
        
    return row.replace('-lat-lon', '')


err_df['lr_path'] = err_df['lr_path'].apply(clean_lr_path)

In [308]:
preds_with_err = plot_preds_df.merge(err_df.set_index('lr_path'), left_index=True, right_index=True)
preds_with_err.to_csv('../../data/model_preds_with_train_test_errors.csv')

In [311]:
pd.set_option('display.max_rows', 500)
preds_with_err.sample(20)

,Co_pred_lr,Co_pred_rf,Co_true,lr_train_mse,lr_test_mse,rf_train_mse,rf_test_mse
Ni,225.888882,511.947501,600.0,1.002014,0.982939,0.062421,0.425724
Ni,267.655503,662.625013,1000.0,1.002014,0.982939,0.062421,0.425724
Ni,244.458213,520.332054,700.0,1.002014,0.982939,0.062842,0.421492
Cu-Ni-Pd-Pt,577.628295,494.833342,820.0,0.597208,0.538930,0.044599,0.299663
Ni,262.885401,470.745605,900.0,1.002014,0.982939,0.062421,0.425724
Ni,254.205106,639.522767,600.0,1.002014,0.982939,0.062551,0.427272
Cu-Ni,753.072269,965.725809,1500.0,0.951613,0.866928,0.056927,0.403983
Ni,250.169364,587.796472,900.0,1.002014,0.982939,0.062842,0.421492
Ni,217.051628,456.056405,900.0,1.002014,0.982939,0.062551,0.427272
Ni,265.242588,547.627861,700.0,1.002014,0.982939,0.062551,0.427272
